In [2]:
import os
import wget
import argparse
import pm4py
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import metrics 

from processtransformer import constants
from processtransformer.models import transformer
from processtransformer.data.loader import LogsDataLoader
from processtransformer.data.processor import LogsDataProcessor


In [3]:
import warnings
warnings.filterwarnings("ignore")

In [10]:
input_file_path= "BPI_Challenge_2012_W.xes.gz"
output_file_path='./datasets/BPIC12W.csv'

#Write to Pandas Dataframe
log = pm4py.read_xes(input_file_path) #Input Filename
df = pm4py.convert_to_dataframe(log)
df.to_csv(output_file_path)

parsing log, completed traces ::   0%|          | 0/9658 [00:00<?, ?it/s]

In [4]:
df

,org:resource,lifecycle:transition,concept:name,time:timestamp,case:REG_DATE,case:concept:name,case:AMOUNT_REQ
0,112,COMPLETE,A_SUBMITTED,2011-09-30 22:38:44.546000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000
1,112,COMPLETE,A_PARTLYSUBMITTED,2011-09-30 22:38:44.880000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000
2,112,COMPLETE,A_PREACCEPTED,2011-09-30 22:39:37.906000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000
3,112,SCHEDULE,W_Completeren aanvraag,2011-09-30 22:39:38.875000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000
4,NaN,START,W_Completeren aanvraag,2011-10-01 09:36:46.437000+00:00,2011-09-30 22:38:44.546000+00:00,173688,20000
...,...,...,...,...,...,...,...
262195,112,COMPLETE,A_PARTLYSUBMITTED,2012-02-29 22:51:17.423000+00:00,2012-02-29 22:51:16.799000+00:00,214376,15000
262196,112,SCHEDULE,W_Afhandelen leads,2012-02-29 22:52:01.287000+00:00,2012-02-29 22:51:16.799000+00:00,214376,15000
262197,11169,START,W_Afhandelen leads,2012-03-01 08:26:46.736000+00:00,2012-02-29 22:51:16.799000+00:00,214376,15000
262198,11169,COMPLETE,A_DECLINED,2012-03-01 08:27:37.118000+00:00,2012-02-29 22:51:16.799000+00:00,214376,15000


In [11]:
data_processor = LogsDataProcessor(name='BPIC12W', filepath="datasets/BPIC12W.csv",  
                                    columns = ["case:concept:name", "concept:name", "time:timestamp"], #specify the columns name containing case_id, activity name and timestamp 
                                    dir_path='datasets', pool = 4)
data_processor.process_logs(task=constants.Task.NEXT_ACTIVITY, sort_temporally= False)

In [12]:
# Load data
data_loader = LogsDataLoader(name = 'BPIC12W')

(train_df, test_df, x_word_dict, y_word_dict, max_case_length, 
    vocab_size, num_output) = data_loader.load_data(constants.Task.NEXT_ACTIVITY)

# Prepare training examples for next activity prediction task
train_token_x, train_token_y = data_loader.prepare_data_next_activity(train_df, 
    x_word_dict, y_word_dict, max_case_length)

In [13]:
learning_rate = 0.001
batch_size = 12
epochs = 10

In [14]:
# Create and train a transformer model
transformer_model = transformer.get_next_activity_model(
    max_case_length=max_case_length, 
    vocab_size=vocab_size,
    output_dim=num_output)

transformer_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

transformer_model.fit(train_token_x, train_token_y, 
    epochs=epochs, batch_size=batch_size)

Epoch 1/10
10896/10896 [==============================] - 1024s 93ms/step - loss: 0.3282 - sparse_categorical_accuracy: 0.8833
Epoch 2/10
10896/10896 [==============================] - 1008s 93ms/step - loss: 0.2718 - sparse_categorical_accuracy: 0.9032
Epoch 3/10
10896/10896 [==============================] - 1007s 92ms/step - loss: 0.2631 - sparse_categorical_accuracy: 0.9046
Epoch 4/10
10896/10896 [==============================] - 1008s 93ms/step - loss: 0.2579 - sparse_categorical_accuracy: 0.9050
Epoch 5/10
10896/10896 [==============================] - 1128s 104ms/step - loss: 0.2551 - sparse_categorical_accuracy: 0.9052
Epoch 6/10
10896/10896 [==============================] - 1147s 105ms/step - loss: 0.2533 - sparse_categorical_accuracy: 0.9051
Epoch 7/10
10896/10896 [==============================] - 1220s 112ms/step - loss: 0.2519 - sparse_categorical_accuracy: 0.9054
Epoch 8/10
10896/10896 [==============================] - 1209s 111ms/step - loss: 0.2492 - sparse_categoric

In [15]:
# Evaluate over all the prefixes (k) and save the results
k, accuracies,fscores, precisions, recalls = [],[],[],[],[]
for i in range(max_case_length):
    test_data_subset = test_df[test_df["k"]==i]
    if len(test_data_subset) > 0:
        test_token_x, test_token_y = data_loader.prepare_data_next_activity(test_data_subset, 
            x_word_dict, y_word_dict, max_case_length)   
        y_pred = np.argmax(transformer_model.predict(test_token_x), axis=1)
        accuracy = metrics.accuracy_score(test_token_y, y_pred)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(
            test_token_y, y_pred, average="weighted")
        k.append(i)
        accuracies.append(accuracy)
        fscores.append(fscore)
        precisions.append(precision)
        recalls.append(recall)

k.append(i + 1)
accuracies.append(np.mean(accuracy))
fscores.append(np.mean(fscores))
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))

1/1 [==============================] - 0s 59ms/step


In [16]:
print('Average accuracy across all prefixes:', np.mean(accuracies))
print('Average f-score across all prefixes:', np.mean(fscores))
print('Average precision across all prefixes:', np.mean(precisions))
print('Average recall across all prefixes:', np.mean(recalls))

Average accuracy across all prefixes: 0.9132852069629318
Average f-score across all prefixes: 0.9057917405729683
Average precision across all prefixes: 0.9074491083984463
Average recall across all prefixes: 0.9123912400244053


In [17]:
data_processor = LogsDataProcessor(name='BPIC12W', filepath="datasets/BPIC12W.csv",  
                                    columns = ["case:concept:name", "concept:name", "time:timestamp"],  #specify the columns name containing case_id, activity name and timestamp 
                                    dir_path='datasets', pool = 4)
data_processor.process_logs(task=constants.Task.NEXT_TIME, sort_temporally= False)

In [18]:
# Load data
data_loader = LogsDataLoader(name = 'BPIC12W')

(train_df, test_df, x_word_dict, y_word_dict, max_case_length, 
    vocab_size, num_output) = data_loader.load_data(constants.Task.NEXT_TIME)

# Prepare training examples for next time prediction task
(train_token_x, train_time_x, train_token_y, time_scaler, y_scaler) = \
                                    data_loader.prepare_data_next_time(train_df, x_word_dict, max_case_length)

In [19]:
learning_rate = 0.001
batch_size = 12
epochs = 1

In [20]:
# Create and train a transformer model
transformer_model = transformer.get_next_time_model(
        max_case_length=max_case_length, 
        vocab_size=vocab_size)

transformer_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
        loss=tf.keras.losses.LogCosh())

transformer_model.fit([train_token_x, train_time_x], train_token_y, 
        epochs=epochs, batch_size=batch_size, verbose=2)

11540/11540 - 998s - loss: 0.1850 - 998s/epoch - 86ms/step


In [21]:
# Evaluate over all the prefixes (k) and save the results
k, maes, mses, rmses = [],[],[],[]
for i in range(max_case_length):
    test_data_subset = test_df[test_df["k"]==i]
    if len(test_data_subset) > 0:
        test_token_x, test_time_x, test_y, _, _ = data_loader.prepare_data_next_time(
            test_data_subset, x_word_dict, max_case_length, time_scaler, y_scaler, False)   

        y_pred = transformer_model.predict([test_token_x, test_time_x])
        _test_y = y_scaler.inverse_transform(test_y)
        _y_pred = y_scaler.inverse_transform(y_pred)

        k.append(i)
        maes.append(metrics.mean_absolute_error(_test_y, _y_pred))
        mses.append(metrics.mean_squared_error(_test_y, _y_pred))
        rmses.append(np.sqrt(metrics.mean_squared_error(_test_y, _y_pred)))

k.append(i + 1)
maes.append(np.mean(maes))
mses.append(np.mean(mses))
rmses.append(np.mean(rmses))  

1/1 [==============================] - 0s 77ms/step


In [22]:
print('Average MAE across all prefixes:', np.mean(maes))
print('Average MSE across all prefixes:', np.mean(mses))
print('Average RMSE across all prefixes:', np.mean(rmses))

Average MAE across all prefixes: 0.43285322
Average MSE across all prefixes: 1.3049976
Average RMSE across all prefixes: 0.92015123


In [24]:
data_processor = LogsDataProcessor(name='BPIC12W', filepath="datasets/BPIC12W.csv",  
                                    columns = ["case:concept:name", "concept:name", "time:timestamp"],  #specify the columns name containing case_id, activity name and timestamp 
                                    dir_path='datasets', pool = 4)
data_processor.process_logs(task=constants.Task.REMAINING_TIME, sort_temporally= False)

In [25]:
# Load data
data_loader = LogsDataLoader(name = 'BPIC12W')

(train_df, test_df, x_word_dict, y_word_dict, max_case_length, 
    vocab_size, num_output) = data_loader.load_data(constants.Task.REMAINING_TIME)

# Prepare training examples for next time prediction task
(train_token_x, train_time_x, 
    train_token_y, time_scaler, y_scaler) = data_loader.prepare_data_remaining_time(train_df, 
    x_word_dict, max_case_length)

In [26]:
learning_rate = 0.001
batch_size = 12
epochs = 1

In [27]:
# Create and train a transformer model
transformer_model = transformer.get_remaining_time_model(
    max_case_length=max_case_length, 
    vocab_size=vocab_size)

transformer_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
    loss=tf.keras.losses.LogCosh())

transformer_model.fit([train_token_x, train_time_x], train_token_y, 
        epochs=epochs, batch_size=batch_size, verbose=2)

11540/11540 - 1033s - loss: 0.2832 - 1033s/epoch - 90ms/step


In [28]:
# Evaluate over all the prefixes (k) and save the results
k, maes, mses, rmses = [],[],[],[]
for i in range(max_case_length):
    test_data_subset = test_df[test_df["k"]==i]
    if len(test_data_subset) > 0:
        test_token_x, test_time_x, test_y, _, _ = data_loader.prepare_data_remaining_time(
            test_data_subset, x_word_dict, max_case_length, time_scaler, y_scaler, False)   

        y_pred = transformer_model.predict([test_token_x, test_time_x])
        _test_y = y_scaler.inverse_transform(test_y)
        _y_pred = y_scaler.inverse_transform(y_pred)

        k.append(i)
        maes.append(metrics.mean_absolute_error(_test_y, _y_pred))
        mses.append(metrics.mean_squared_error(_test_y, _y_pred))
        rmses.append(np.sqrt(metrics.mean_squared_error(_test_y, _y_pred)))

k.append(i + 1)
maes.append(np.mean(maes))
mses.append(np.mean(mses))
rmses.append(np.mean(rmses))  

1/1 [==============================] - 0s 64ms/step


In [29]:
print('Average MAE across all prefixes:', np.mean(maes))
print('Average MSE across all prefixes:', np.mean(mses))
print('Average RMSE across all prefixes:', np.mean(rmses))

Average MAE across all prefixes: 5.1657896
Average MSE across all prefixes: 39.333805
Average RMSE across all prefixes: 6.090715
